# Module 1 - Explore Graph with Cypher

This module has the following objective:
- Running Cypher queries to better understand the data we've loaded

Import our usual suspects (and some more...)

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv
from graphdatascience import GraphDataScience
from neo4j import Query, GraphDatabase, RoutingControl, Result

# Setup

Load env variables

In [3]:
env_file = 'ws.env'

In [4]:
if os.path.exists(env_file):
    load_dotenv(env_file, override=True)

    # Neo4j
    HOST = os.getenv('NEO4J_URI')
    USERNAME = os.getenv('NEO4J_USERNAME')
    PASSWORD = os.getenv('NEO4J_PASSWORD')
    DATABASE = os.getenv('NEO4J_DATABASE')
else:
    print(f"File {env_file} not found.")

Setup connection to the database with the [Python Driver](https://neo4j.com/docs/python-manual/5/).

In [5]:
driver = GraphDatabase.driver(
    HOST,
    auth=(USERNAME, PASSWORD)
)

Test the connection

In [6]:
driver.execute_query(
    """
    MATCH (n) RETURN COUNT(n) as Count
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

,Count
0,1388


## Explore the graph 

Now to the database and observe what is there. Example queries:

- `MATCH (n:Customer) RETURN n LIMIT 25`

- `MATCH (n:Chunk) RETURN n LIMIT 25`

- `MATCH p=()-[r:PART_OF]->() RETURN p LIMIT 25`

We can also run this via the [Neo4j Python Driver](https://neo4j.com/docs/python-manual/5/) wihtin your Notebook. Let's do so below!

### How many Customers are there in the graph?

In [7]:
driver.execute_query(
    """
    MATCH (cust:Customer) 
    RETURN COUNT(cust)
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

,COUNT(cust)
0,50


### What products do customers have? 

In [8]:
customer_product_df = driver.execute_query(
    """
    MATCH (cust:Customer)-[:HAS_PRODUCT]->(prod:Product)
    WITH DISTINCT cust, COLLECT(prod.name) AS products
    RETURN cust.name AS customer_name, SIZE(products) AS num_products, products LIMIT 10
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

In [9]:
customer_product_df

,customer_name,num_products,products
0,Lucas Vos,1,[SpaarRekening Product]
1,Rik Meijer,1,[RaboBusiness Banking Product]
2,Sara Bakker,2,"[DirectRekening Product, SpaarRekening Product]"
3,Jan Smit,3,"[BeleggersRekening Product, Kortlopende Reis P..."
4,Jan De Vries,1,[BeleggersRekening Product]
5,Henk De Vries,1,[DirectRekening Product]
6,Eva Bos,2,"[RaboBusiness Banking Product, Kortlopende Rei..."
7,Jan Smit,3,"[RaboBusiness Banking Product, BeleggersRekeni..."
8,Lars Vos,2,"[BeleggersRekening Product, Kortlopende Reis P..."
9,Bram Smit,2,"[SpaarRekening Product, BeleggersRekening Prod..."


### How many products on average?

In [10]:
customer_product_df["num_products"].describe()

count    10.000000
mean      1.800000
std       0.788811
min       1.000000
25%       1.000000
50%       2.000000
75%       2.000000
max       3.000000
Name: num_products, dtype: float64

### What are the most common products?

In [11]:
product_df = driver.execute_query(
    """
    MATCH (cust:Customer)-[:HAS_PRODUCT]->(prod:Product)
    RETURN DISTINCT prod.name AS product_name, COUNT(cust) AS num_customers ORDER BY num_customers DESC
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

In [12]:
product_df

,product_name,num_customers
0,Kortlopende Reis Product,22
1,SpaarRekening Product,21
2,DirectRekening Product,18
3,BeleggersRekening Product,18
4,RaboBusiness Banking Product,14


### What documents are in the database?

In [13]:
document_df = driver.execute_query(
    """
    MATCH (prod_type:ProductType)<-[:RELATED_TO]-(doc:Document) 
    RETURN DISTINCT prod_type.name AS product_type, doc.file_name AS document, doc.pages AS num_pages
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

In [14]:
document_df

,product_type,document,num_pages
0,Kortlopende Reis,Interpolis Short-Term Travel Insurance.pdf,22
1,SpaarRekening,Rabo SpaarRekening 2020.pdf,14
2,RaboBusiness Banking,Terms & Conditions for Online Business Service...,29
3,DirectRekening,Payment and Online Services Terms Sept 2022.pdf,80
4,BeleggersRekening,Rabo Beleggersrekening Terms 2020.pdf,25


### How many chunks are the documents split into?

In [15]:
document_chunks_df = driver.execute_query(
    """
    MATCH (doc:Document)<-[:PART_OF]-(chunk:Chunk)
    WITH DISTINCT doc, COLLECT(chunk) AS chunks
    RETURN doc.file_name AS file_name, doc.pages AS pages, SIZE(chunks) AS chunk_count
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

In [16]:
document_chunks_df

,file_name,pages,chunk_count
0,Interpolis Short-Term Travel Insurance.pdf,22,46
1,Rabo SpaarRekening 2020.pdf,14,44
2,Terms & Conditions for Online Business Service...,29,88
3,Payment and Online Services Terms Sept 2022.pdf,80,354
4,Rabo Beleggersrekening Terms 2020.pdf,25,78


## Extracted Definitions

Let's dive into the extracted definitions. 

In [ ]:
definitions_df = driver.execute_query(
    """
    MATCH (doc:Document)<-[:PART_OF]-(chunk:Chunk)-[:MENTIONS]->(def:Definition)
    WITH DISTINCT doc, COLLECT(def) AS definitions
    RETURN doc.file_name AS file_name, SIZE(definitions) AS definition_count, [def in definitions | def.term][0] AS definition_example, [def in definitions | def.description][0] AS description_example
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

In [ ]:
definitions_df

,file_name,definition_count,definition_example,description_example
0,Interpolis Short-Term Travel Insurance.pdf,451,terms and conditions,The rules and guidelines that may be changed i...
1,Rabo SpaarRekening 2020.pdf,570,rabo,Savings Account 2020
2,Terms & Conditions for Online Business Service...,980,online service,Services that allow you to conduct (banking) b...
3,Payment and Online Services Terms Sept 2022.pdf,4523,payment,Any payment transaction resulting from the pay...
4,Rabo Beleggersrekening Terms 2020.pdf,1050,rabo,Savings Account 2020


### What definitions are mentioned most frequently within chunks?

Some definitions are mentioned in many chunks. These "supernodes" are often not informative as they do not capture specific information about the particular chunk itself.

In [ ]:
definition_count_df = driver.execute_query(
    """
    MATCH (doc:Document)<-[:PART_OF]-(chunk:Chunk)-[:MENTIONS]->(def:Definition)
    WITH DISTINCT def, COUNT(DISTINCT chunk) AS chunk_count, COUNT(DISTINCT doc) AS document_count
    RETURN def.term AS definition, def.description AS description, chunk_count, document_count ORDER BY chunk_count DESC LIMIT 25
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df(),
)

In [ ]:
definition_count_df

,definition,description,chunk_count,document_count
0,you,"the account holder, and/or all account holders...",524,5
1,we,The Coöperatieve Rabobank U.A.,483,5
2,account,The payment account you hold with us.,280,4
3,app,A software application that you must check if ...,275,5
4,payment,Any payment transaction resulting from the pay...,251,5
5,for example,shall be construed as for example without limi...,232,5
6,credit,Includes a construction deposit and conditions...,220,3
7,rabo,Savings Account 2020,205,4
8,online service,Services that allow you to conduct (banking) b...,174,3
9,agreement,the agreement concluded between you and us in ...,142,5
